In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고가 뜨지 않도록..
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 출력 창 청소
from IPython.display import clear_output

import time

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 딥러닝
import tensorflow as tf

# 딥러닝 모델 구조를 정의하는 것
from tensorflow.keras.models import Sequential
# 층 모델(은닉층과 출력층)
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 것
from tensorflow.keras.layers import Activation
# CNN : 커널을 통해 합성곱을 구하는 것. 이미지의 특징을 두드러지게 한다.
from tensorflow.keras.layers import Conv2D
# MaxPooling : 불필요한 부분을 제거하는 역할을 한다
from tensorflow.keras.layers import MaxPool2D
# Flatten : CNN, MaxPooling은 2차원 데이터를 사용하고 출력하기 때문에
# 1차원 데이터로 변하는 것을 사용해야 한다.
from tensorflow.keras.layers import Flatten
# Dropout : 노드의 일부를 on/off 하면서 과적합을 예방한다.
from tensorflow.keras.layers import Dropout
# Embedding : 주어진 단어 데이터를 원핫 인코딩하고 차원축소를 한다.
from tensorflow.keras.layers import Embedding
# 문장 학습을 위한 RNN(LSTM)
from tensorflow.keras.layers import LSTM
# Cnn1D
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPool1D


# 데이터 표준화
from sklearn.preprocessing import StandardScaler
# 문자열을 숫자로 변환
from sklearn.preprocessing import LabelEncoder
# 원핫 인코딩
from tensorflow.keras.utils import to_categorical

# 학습데이터와 검증데이터로 나눈다
from sklearn.model_selection import train_test_split

# K폴드 교차검증
from sklearn.model_selection import KFold

# 저장된 모델을 복원한다.
from tensorflow.keras.models import load_model

# epoch 마다 저장을 하기 위해 사용한다.
from tensorflow.keras.callbacks import ModelCheckpoint
# 조기 중단을 위해 사용한다.
from tensorflow.keras.callbacks import EarlyStopping

# 문장을 잘라준다.
from tensorflow.keras.preprocessing.text import Tokenizer
# 모든 문장의 데이터의 단어 데이터 수를 동일한 수로 맞춰준다.
from tensorflow.keras.preprocessing.sequence import pad_sequences
# 문자열을 단어사전으로 만들어준다.
from tensorflow.keras.preprocessing.text import text_to_word_sequence

import os
import glob
from PIL import Image


# GPU 할당
gpus = tf.config.experimental.list_physical_devices('GPU')
# GPU가 있다면..
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)

In [2]:
# 폴더 목록을 가져온다.
root_dir = '101_ObjectCategories'

a1 = os.walk(root_dir)
categories = list(a1)[0][1]

# 수업 실습을 위해 (나중에 주석처리해서 테스트)
categories = categories[:5]

categories


['accordion', 'airplanes', 'anchor', 'ant', 'BACKGROUND_Google']

In [3]:
# 카테고리의 개수
nb_classes = len(categories)
nb_classes


5

In [4]:
# 이미지의 크기
image_w = 64
image_h = 64


In [5]:
# 이미지 데이터와 결과 데이터를 담을 리스트
X = []
y = []


In [6]:
# 폴더의 수 만큼 반복한다.
for idx, cat in enumerate(categories) :
    print(f'{cat} 작업중')
    
    # 폴더명을 붙힌 경로를 생성한다.
    image_dir = root_dir + '/' + cat
    # print(image_dir)
    
    # 해당 폴더에서 파일 이름의 목록을 가져온다
    files = glob.glob(image_dir + '/*.jpg') # 확장자가 jpg인것만
    # print(files)
    
    # 파일의 수 만큼 반복한다.
    for i, f in enumerate(files) :
        # 이미지 파일의 데이터를 가져온다.
        img = Image.open(f)
        # 색상 체계 변경(CNN을 위해..)
        img = img.convert('RGB')
        # 이미지 크기 변경
        img = img.resize((image_w, image_h))
        # 데이터를 ndarray로 변환한다.
        data = np.asarray(img)
        # 데이터를 담는다.
        X.append(data)
        y.append(idx)
        
        # -20도 에서 20도까지 5도씩 증가하며 회전한다.
        for angle in range(-20, 20, 5) :
            # 이미지 회전
            img2 = img.rotate(angle)
            
            data = np.asarray(img2)
            X.append(data)
            y.append(idx)
            
            # 회전한 이미지를 반전시킨다.
            img3 = img2.transpose(Image.FLIP_LEFT_RIGHT)
            
            data = np.asarray(img3)
            X.append(data)
            y.append(idx)





accordion 작업중
airplanes 작업중
anchor 작업중
ant 작업중
BACKGROUND_Google 작업중


In [7]:
# 전체 이미지의 수
len(X)

23902

In [8]:
X = np.array(X)
y = np.array(y)

In [9]:
# 학습 데이터와 검증데이터로 나눠 저장한다.
X_train, X_test, y_train, y_test = train_test_split(X, y)

xy = (X_train, X_test, y_train, y_test)

np.save('101_new2.npy', xy)
print('저장완료')


저장완료
